In [ ]:
# sample = {
#     "question": "10% of 11.5m2 is:",
#     "choices": ["A. 10,15dm2", "B. 1,5m2", "C. 15,5m2", "D. 1,15m2"],
# }

example_sample = {
    "id": "xxxx",


    
    # "question": "A store sold 30% of its existing goods and earned 15,000,000 VND. If all goods were sold, how much money would the store earn?",
    # "choices": [
    #     "A. 4 500 000 VND",
    #     "B. 45 000 000 VND",
    #     "C. 50 000 000 VND",
    #     "D. 450 000 000 VND"
    # ]
    # "question": "8 dm2 24 cm2 = ……… dm2. The appropriate number to fill in the blanks is:",
    #       "choices": [
    #          "A. 824",
    #          "B. 82.4",
    #          "C. 8.24",
    #          "D. 0.824"
    #       ],
    
#     "question": "2 ${\\times}$ 9 ? – 28 5 ${\\times}$ 3",
#     "choices": [
#     "A. 52",
#     "B. 53",
#     "C. 41",
#     "D. 45"
#     ],
    
#     "question": "The appropriate number to fill in the blanks 5kg 30g = …….. kg is:",
#           "choices": [
#              "A. 53",
#              "B. 50.3",
#              "C. 5.03",
#              "D. 5.3"
#           ]

    "question": "A cyclist rode from A at 7 o'clock at a speed of 12km/h. At 8 o'clock a motorcyclist also from A chased the cyclist at a speed of 42km/h. Ask the cyclist What time did the machine catch up with the cyclist?",
          "choices": [
             "A. 24 minutes",
             "B. 1 hour",
             "C. 7 hours 24 minutes",
             "D. 8 hours 24 minutes"
          ]
    # "question": "Một cửa hàng đã bán 30% số hàng hiện có và thu được 15 000 000 đồng. Hỏi nếu bán hết hàng thì cửa hàng thu được bao nhiêu tiền?",
    #     "choices": [
    #         "A. 4 500 000 đồng",
    #         "B. 45 000 000 đồng",
    #         "C. 50 000 000 đồng",
    #         "D. 450 000 000 đồng"
    #     ]
}

answer(example_sample, call_mistral_math, debug=True)

In [ ]:
import torch
import gc
import json
from llama_cpp import Llama
from huggingface_hub import hf_hub_download
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from langdetect import detect
from sentence_transformers import SentenceTransformer
from transformers import AutoModelForCausalLM

import numpy as np

CACHE_DIR = "./cache"

dict_map = {
    "òa": "oà",
    "Òa": "Oà",
    "ÒA": "OÀ",
    "óa": "oá",
    "Óa": "Oá",
    "ÓA": "OÁ",
    "ỏa": "oả",
    "Ỏa": "Oả",
    "ỎA": "OẢ",
    "õa": "oã",
    "Õa": "Oã",
    "ÕA": "OÃ",
    "ọa": "oạ",
    "Ọa": "Oạ",
    "ỌA": "OẠ",
    "òe": "oè",
    "Òe": "Oè",
    "ÒE": "OÈ",
    "óe": "oé",
    "Óe": "Oé",
    "ÓE": "OÉ",
    "ỏe": "oẻ",
    "Ỏe": "Oẻ",
    "ỎE": "OẺ",
    "õe": "oẽ",
    "Õe": "Oẽ",
    "ÕE": "OẼ",
    "ọe": "oẹ",
    "Ọe": "Oẹ",
    "ỌE": "OẸ",
    "ùy": "uỳ",
    "Ùy": "Uỳ",
    "ÙY": "UỲ",
    "úy": "uý",
    "Úy": "Uý",
    "ÚY": "UÝ",
    "ủy": "uỷ",
    "Ủy": "Uỷ",
    "ỦY": "UỶ",
    "ũy": "uỹ",
    "Ũy": "Uỹ",
    "ŨY": "UỸ",
    "ụy": "uỵ",
    "Ụy": "Uỵ",
    "ỤY": "UỴ",
    }

In [ ]:
tokenizer_vi2en = AutoTokenizer.from_pretrained(
    "vinai/vinai-translate-vi2en", src_lang="vi_VN", cache_dir="./cache"
)
model_vi2en = AutoModelForSeq2SeqLM.from_pretrained(
    "vinai/vinai-translate-vi2en", cache_dir="./cache"
)

In [ ]:
model_name = "meta-math/MetaMath-Mistral-7B"
tokenizer = AutoTokenizer.from_pretrained(
    model_name, 
    cache_dir="./cache",
)
backbone_model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    device_map="auto", 
    torch_dtype=torch.bfloat16,
    cache_dir="./cache",
)

In [ ]:
def call_mistral_math(input_str, **kwargs):
    model_inputs = tokenizer([input_str], return_tensors="pt")
    model_inputs = {k: v.to(backbone_model.device) for k, v in model_inputs.items()}
    model_outputs = backbone_model.generate(
        **model_inputs,
        do_sample=False,
        max_new_tokens=1000,
    )
    return tokenizer.batch_decode(model_outputs, skip_special_tokens=True)[0]

In [ ]:
def translate_vi2en(vi_text: str) -> str:
    for i, j in dict_map.items():
        vi_text = vi_text.replace(i, j)
    input_ids = tokenizer_vi2en(vi_text, return_tensors="pt").input_ids
    output_ids = model_vi2en.generate(
        input_ids,
        decoder_start_token_id=tokenizer_vi2en.lang_code_to_id["en_XX"],
        num_return_sequences=1,
        # # With sampling
        # do_sample=True,
        # top_k=100,
        # top_p=0.8,
        # With beam search
        num_beams=5,
        early_stopping=True
    )
    en_text = tokenizer_vi2en.batch_decode(output_ids, skip_special_tokens=True)
    en_text = " ".join(en_text)
    return en_text

translate_vi2en("Một cửa hàng đã bán 30% số hàng hiện có và thu được 15 000 000 đồng.")

In [ ]:
embedding_model = SentenceTransformer('BAAI/bge-large-en-v1.5')

In [ ]:
detect("Một cửa hàng đã bán 30% số hàng hiện có và thu được 15 000 000 đồng")

## Prompt

In [ ]:
PROMPT_DICT = {
    "prompt_input": (
        "Below is an instruction that describes a task, paired with an input that provides further context. "
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:"
    ),
}

INPUT_STR_PROMPT = """Question: {question}
Options:
{choices[0]}
{choices[1]}
{choices[2]}
{choices[3]}"""

def get_instruction_str(sample):
    num_options = len(sample["choices"])
    option_chars = [chr(ord("A") + i) for i in range(num_options)]
    return f"Answer the following question for me by choosing option {', '.join(option_chars[:-1])}, or {option_chars[-1]}."

def get_input_str(sample):
    num_options = len(sample["choices"])
    INPUT_STR_PROMPT = """Question: {question}
Options:
"""
    for i in range(num_options):
        INPUT_STR_PROMPT += "{" + f"choices[{i}]" + "}\n"
    INPUT_STR_PROMPT = INPUT_STR_PROMPT[:-1]
    return INPUT_STR_PROMPT.format(**sample)

In [ ]:
def get_model_str(sample):
    instruction_str = get_instruction_str(sample)
    input_str = get_input_str(sample)
    return PROMPT_DICT["prompt_input"].format(instruction=instruction_str, input=input_str)

In [ ]:
def detect_lang(text):
    try: 
        lang = detect(text)
    except:
        lang = None
    return lang

def process_option(option, lower=False):
    option = option[2:].strip()
    return option.lower() if lower else option

def preprocess_sample(sample, debug=False):
    question = sample["question"]
    choices = sample["choices"]
    choices = [process_option(choice, lower=False) for choice in choices]
    
    if detect_lang(question) != "en":
        if debug: print(f"Translating question: {question}")
        question = translate_vi2en(question)
    
    for i, choice in enumerate(choices):
        if debug: print(f"Translating choice {i}: {choice}")
        if detect_lang(choice) != "en":
            choices[i] = translate_vi2en(choice)
    
    choices = [chr(ord("A") + i) + ". " + choice for i, choice in enumerate(choices)]
    
    return {
        "question": question,
        "choices": choices
    }

def post_process_output(output, sample):
    options = sample["choices"]
    options = [process_option(option, lower=True) for option in options]
    
    option_id = {o:i for i, o in enumerate(options)}
    
    idx = -1
    for option in sorted(options, key=len, reverse=True):
        if option in output:
            idx = option_id[option]

    if idx == -1:
        pred = output.split("\n")[-3:]
        pred = " ".join(pred)
        choices = sample["choices"]
        embeddings_1 = embedding_model.encode(pred, normalize_embeddings=True)
        embeddings_2 = embedding_model.encode(choices, normalize_embeddings=True)
        similarity = embeddings_1 @ embeddings_2.T
        idx = np.argmax(similarity)
    return idx
    
def answer(sample, math_llm, num_trials=2, debug=False):
    processed_sample = preprocess_sample(sample, debug)
    
    model_str = get_model_str(processed_sample)
    if debug: print(model_str)
    
    for i in range(num_trials):
        math_output = math_llm(
            model_str,
            top_k=1,
            max_tokens=1024,
        )
        try:
            output = math_output["choices"][0]["text"].strip()
        except:
            output = math_output
        if output != "" and i > 0:
            if debug: print(output)
            break

    idx = post_process_output(output, processed_sample)
    result = {
        "id" : sample["id"],
        "answer": sample["choices"][idx]
    }

    if debug: print(result)
    
    return result, output

## Inference

In [ ]:
test_samples = json.load(open("./datasets/public_test/math_test.json", "r", encoding="utf-8"))['data']
test_samples[3]

In [ ]:
answer(test_samples[3], math_model, debug=True)

In [ ]:
import random
from tqdm import tqdm

answer_data = []
logging_data = []
for sample in tqdm(test_samples):
    debug = True if random.random() < 0.1 else False
    ans, model_output = answer(sample, math_model, debug=debug)
    if debug:
        print("*" * 50 + "\n\n")
    answer_data.append(ans)

    sample["model_output"] = model_output
    logging_data.append(sample)

In [ ]:
import pandas as pd

submission_df = pd.DataFrame(answer_data)
submission_df.to_csv("./submissions/baseline1.csv", index=False)

In [ ]:
with open("./logs/baseline1.json", "w", encoding="utf-8") as f:
    json.dump(logging_data, f, ensure_ascii=False, indent=4)